In [29]:
import pickle as pkl
import pandas as pd

In [42]:
with open('data/iemocap/data_iemocap.pkl', 'rb') as f:
    x = pkl.load(f)
    
x.keys()

dict_keys(['train', 'dev', 'test'])

In [32]:
df = pd.read_pickle("data/iemocap_gc/data_iemocap_gc.pkl")
type(df)

dict

In [38]:
import torch
import math
import numpy as np
import random
import pickle
from tqdm import tqdm
# from utils import set_seed

def load_iemocap():
    path = "data/iemocap_roberta/iemocap_roberta.pkl"
    with open(path, "rb") as f:
        unsplit = pickle.load(f)
    
    speaker_to_idx = {"M": 0, "F": 1}

    data = {
        "train": [], "dev": [], "test": [],
    }
    trainVid = list(unsplit["trainVid"])
    random.shuffle(trainVid)
    testVid = list(unsplit["testVid"])

    dev_size = int(len(trainVid) * 0.1)
    
    spliter = {
        "train": trainVid[dev_size:],
        "dev": trainVid[:dev_size],
        "test": testVid
    }

    for split in data:
        for uid in tqdm(spliter[split], desc=split):
            data[split].append(
                {
                    "uid" : uid,
                    "speakers" : [speaker_to_idx[speaker] for speaker in unsplit["speaker"][uid]],
                    "labels" : unsplit["label"][uid],
                    "text": unsplit["text"][uid],
                    "audio": unsplit["audio"][uid],
                    "visual": unsplit["visual"][uid],
                    "sentence" : unsplit["sentence"][uid],
                }
            )
    
    return data

class Dataloader:
    def __init__(self, data, args):
        self.data = data
        self.batch_size = args.batch_size
        self.num_batches = math.ceil(len(data)/ self.batch_size)
        self.dataset = args.dataset
        self.embedding_dim = args.embedding_dim[self.dataset]
    
    def __len__(self):
        return self.num_batches
    
    def __getitem__(self, index):
        batch = self.raw_batch(index)
        return self.padding(batch)

    def raw_batch(self, index):
        assert index < self.num_batches, "batch_idx %d > %d" % (index, self.num_batches)
        batch = self.data[index * self.batch_size : (index + 1) * self.batch_size]
        return batch

    def padding(self, samples):
        batch_size = len(samples)
        text_len_tensor = torch.tensor([len(s["text"]) for s in samples]).long()
        mx = torch.max(text_len_tensor).item()
        
        audio_tensor = torch.zeros((batch_size, mx, self.embedding_dim['a']))
        text_tensor = torch.zeros((batch_size, mx, self.embedding_dim['t']))
        visual_tensor = torch.zeros((batch_size, mx, self.embedding_dim['v']))

        speaker_tensor = torch.zeros((batch_size, mx)).long()

        labels = []
        utterances = []
        for i, s in enumerate(samples):
            cur_len = len(s["text"])
            utterances.append(s["sentence"])

            tmp_t = []
            tmp_a = []
            tmp_v = []
            for t, a, v in zip(s["text"], s["audio"], s["visual"]):
                tmp_t.append(torch.tensor(t))
                tmp_a.append(torch.tensor(a))
                tmp_v.append(torch.tensor(v))
                
            tmp_a = torch.stack(tmp_a)
            tmp_t = torch.stack(tmp_t)
            tmp_v = torch.stack(tmp_v)

            text_tensor[i, :cur_len, :] = tmp_t
            audio_tensor[i, :cur_len, :] = tmp_a
            visual_tensor[i, :cur_len, :] = tmp_v
            
            speaker_tensor[i, :cur_len] = torch.tensor(s["speakers"])

            labels.extend(s["labels"])

        label_tensor = torch.tensor(labels).long()
        

        data = {
            "length": text_len_tensor,
            "tensor": {
                "t": text_tensor,
                "a": audio_tensor,
                "v": visual_tensor,
            },
            "speaker_tensor": speaker_tensor,
            "label_tensor": label_tensor,
            "utterance_texts": utterances,
        }

        return data

    def shuffle(self):
        random.shuffle(self.data)



data = load_iemocap()
data.keys()

test: 100%|██████████| 31/31 [00:00<00:00, 60335.70it/s]


In [39]:
data.keys()


dict_keys(['train', 'dev', 'test'])

In [43]:
output_path = "data/iemocap_roberta/data_iemocap_roberta.pkl"

# Dump the 'data' dictionary into a pickle file
with open(output_path, "wb") as f:
    pickle.dump(data, f)

print(f"Data has been successfully saved to {output_path}")

Data has been successfully saved to data/iemocap_roberta/data_iemocap_roberta.pkl
